<a href="https://colab.research.google.com/github/bec2148/mnist-finalproject/blob/main/CIFAR10_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Base code https://pytorch.org/tutorials/beginner/knowledge_distillation_tutorial.html

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time

NUM_EPOCHS = 10

# Check if GPU is available, and if not, use the CPU
print("torch.cuda.is_available()", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.is_available() False


In [15]:
# Below we are preprocessing data for CIFAR-10. We use an arbitrary batch size of 128.
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading the CIFAR-10 dataset:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Comment back in for efficiency

# from torch.utils.data import Subset
# num_images_to_keep = 2000
# train_dataset = Subset(train_dataset, range(min(num_images_to_keep, 50_000)))
# test_dataset = Subset(test_dataset, range(min(num_images_to_keep, 10_000)))

In [17]:
#Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

In [18]:
# Deeper neural network class to be used as teacher:
class DeepNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DeepNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Lightweight neural network class to be used as student:
class LightNN(nn.Module):
    def __init__(self, num_classes=10, conv2_feature_count=10, linear_feature_count=128):
        super(LightNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Changed from (16, 16) -> (16, conv2_feature_count)
            nn.Conv2d(16, conv2_feature_count, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            # Changed (1024, 256 -> 512, linear_feature_count)
            nn.Linear(64 * conv2_feature_count, linear_feature_count),
            nn.ReLU(),
            nn.Dropout(0.1),
            # Changed from 256 -> linear_feature_count
            nn.Linear(linear_feature_count, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [19]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs: A collection of batch_size images
            # labels: A vector of dimensionality batch_size with integers denoting class of each image
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # outputs: Output of the network for the collection of images. A tensor of dimensionality batch_size x num_classes
            # labels: The actual labels of the images. Vector of dimensionality batch_size
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def test(model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [20]:
torch.manual_seed(42)
nn_deep = DeepNN(num_classes=10).to(device)
start_teacher_train = time.perf_counter()
train(nn_deep, train_loader, epochs=NUM_EPOCHS, learning_rate=0.001, device=device)
end_teacher_train = time.perf_counter()
teacher_train_seconds = end_teacher_train - start_teacher_train
test_accuracy_deep = test(nn_deep, test_loader, device)

# Instantiate the lightweight network:
torch.manual_seed(42)
nn_light = LightNN(num_classes=10).to(device)

Epoch 1/10, Loss: 1.3612387064472793
Epoch 2/10, Loss: 0.8997144103050232
Epoch 3/10, Loss: 0.7012277718852548
Epoch 4/10, Loss: 0.5637503419538288
Epoch 5/10, Loss: 0.43957434392646144
Epoch 6/10, Loss: 0.33047518240826207
Epoch 7/10, Loss: 0.24322747768800887
Epoch 8/10, Loss: 0.19116751765808487
Epoch 9/10, Loss: 0.15074250264012296
Epoch 10/10, Loss: 0.12778362168280213
Test Accuracy: 74.87%


In [21]:
torch.manual_seed(42)
new_nn_light = LightNN(num_classes=10).to(device)

In [22]:
# Print the norm of the first layer of the initial lightweight model
print("Norm of 1st layer of nn_light:", torch.norm(nn_light.features[0].weight).item())
# Print the norm of the first layer of the new lightweight model
print("Norm of 1st layer of new_nn_light:", torch.norm(new_nn_light.features[0].weight).item())

Norm of 1st layer of nn_light: 2.327361822128296
Norm of 1st layer of new_nn_light: 2.327361822128296


In [23]:
## Print the total number of parameters in each model:

total_params_deep = "{:,}".format(sum(p.numel() for p in nn_deep.parameters()))
print(f"DeepNN parameters: {total_params_deep}")
total_params_light = "{:,}".format(sum(p.numel() for p in nn_light.parameters()))
print(f"LightNN parameters: {total_params_light}")

DeepNN parameters: 1,186,986
LightNN parameters: 85,236


In [24]:
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
#### print(f"Student accuracy: {test_accuracy_light_ce:.2f}%")

Teacher accuracy: 74.87%


In [25]:
## Distillation loss is calculated from the logits of the networks. It only returns gradients to the student:

def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T, soft_target_loss_weight, ce_loss_weight, device):
    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.eval()  # Teacher set to evaluation mode
    student.train() # Student to train mode

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass with the teacher model - do not save gradients here as we do not change the teacher's weights
            with torch.no_grad():
                teacher_logits = teacher(inputs)

            # Forward pass with the student model
            student_logits = student(inputs)

            #Soften the student logits by applying softmax first and log() second
            soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

            # Calculate the soft targets loss. Scaled by T**2 as suggested by the authors of the paper "Distilling the knowledge in a neural network"
            soft_targets_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (T**2)

            # Calculate the true label loss
            label_loss = ce_loss(student_logits, labels)

            # Weighted sum of the two losses
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

# Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
start_student_distill = time.perf_counter()
train_knowledge_distillation(teacher=nn_deep, student=new_nn_light, train_loader=train_loader, epochs=NUM_EPOCHS, learning_rate=0.001, T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75, device=device)
end_student_distill = time.perf_counter()
test_accuracy_light_ce_and_kd = test(new_nn_light, test_loader, device)

# Compare the student test accuracy with and without the teacher, after distillation
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
#### print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
print(f"Student accuracy with CE + KD: {test_accuracy_light_ce_and_kd:.2f}%")

Epoch 1/10, Loss: 2.5100126989052423
Epoch 2/10, Loss: 2.016175822528732
Epoch 3/10, Loss: 1.8027952787516368
Epoch 4/10, Loss: 1.66039234445528
Epoch 5/10, Loss: 1.5470637256836952
Epoch 6/10, Loss: 1.4683849488377876
Epoch 7/10, Loss: 1.3928200309843664
Epoch 8/10, Loss: 1.3370106485493654
Epoch 9/10, Loss: 1.2766336903852575
Epoch 10/10, Loss: 1.225502488405808
Test Accuracy: 67.55%
Teacher accuracy: 74.87%
Student accuracy with CE + KD: 67.55%


In [26]:
for i in range(8):
    # conv2_feature_count:  8..=16
    conv2_feature_count = 8 + i
    # linear_feature_count: 128..=256
    linear_feature_count = 128 + i * 32
    new_nn_light = LightNN(num_classes=10, conv2_feature_count=conv2_feature_count, linear_feature_count=linear_feature_count).to(device)

    # Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
    start_student_distill = time.perf_counter()
    train_knowledge_distillation(teacher=nn_deep, student=new_nn_light, train_loader=train_loader, epochs=NUM_EPOCHS, learning_rate=0.001, T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75, device=device)
    end_student_distill = time.perf_counter()
    student_distill_seconds = end_student_distill - start_student_distill
    test_accuracy_light_kd = test(new_nn_light, test_loader, device)

    print(f"Teacher training time: {teacher_train_seconds:0.4f} seconds")

    ## Print the total number of parameters in each model:
    total_params_deep = "{:,}".format(sum(p.numel() for p in nn_deep.parameters()))
    print(f"DeepNN parameters: {total_params_deep}")
    total_params_light = "{:,}".format(sum(p.numel() for p in new_nn_light.parameters()))
    print(f"LightNN parameters: {total_params_light}")

    print(f"Student distillation time: {student_distill_seconds:0.4f} seconds")

    # Compare the student test accuracy with and without the teacher, after distillation
    print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
    #### print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
    print(f"Student accuracy with knowledge distillation: {test_accuracy_light_kd:.2f}%\n")

Epoch 1/10, Loss: 2.5723260750855936
Epoch 2/10, Loss: 2.0379202170750066
Epoch 3/10, Loss: 1.8533824566380142
Epoch 4/10, Loss: 1.7170400107303239
Epoch 5/10, Loss: 1.6197278377650035
Epoch 6/10, Loss: 1.5336737962025206
Epoch 7/10, Loss: 1.4673652075745565
Epoch 8/10, Loss: 1.4029365216984468
Epoch 9/10, Loss: 1.3527798396547128
Epoch 10/10, Loss: 1.3031982380106015
Test Accuracy: 67.86%
Teacher training time: 980.7731 seconds
DeepNN parameters: 1,186,986
LightNN parameters: 68,562
Student distillation time: 431.6734 seconds
Teacher accuracy: 74.87%
Student accuracy with knowledge distillation: 67.86%

Epoch 1/10, Loss: 2.5684684191823313
Epoch 2/10, Loss: 2.0111613011421143
Epoch 3/10, Loss: 1.8108021484311585
Epoch 4/10, Loss: 1.668108017853154
Epoch 5/10, Loss: 1.559086444432778
Epoch 6/10, Loss: 1.4713312914914183
Epoch 7/10, Loss: 1.408829275604404
Epoch 8/10, Loss: 1.3477377572937694
Epoch 9/10, Loss: 1.2959099873862303
Epoch 10/10, Loss: 1.2407131832274025
Test Accuracy: 67.42